## Problem 3

The engineer explains that an engine part seems to be missing from the engine, but nobody can figure out which one. If you can add up all the part numbers in the engine schematic, it should be easy to work out which part is missing.

The engine schematic (your puzzle input) consists of a visual representation of the engine. There are lots of numbers and symbols you don't really understand, but apparently any number adjacent to a symbol, even diagonally, is a "part number" and should be included in your sum. (Periods (.) do not count as a symbol.)

Here is an example engine schematic:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, two numbers are not part numbers because they are not adjacent to a symbol: 114 (top right) and 58 (middle right). Every other number is adjacent to a symbol and so is a part number; their sum is 4361.

Of course, the actual engine schematic is much larger. What is the sum of all of the part numbers in the engine schematic?



In [387]:
import re

In [395]:
# reading file
txt = open("/Users/adolci/Documents/Python/private/advent_of_code/2023/problem_3_input.txt", 'r').read()

# split into rows and remove the last element (empty)
txt = txt.split("\n")[:-1]

In [396]:
# add first and last rows full of dots -->
# --> in this way you can generalise without a different logic for first and last row
empty_txt = "." * len(txt[0])
txt.insert(0, empty_txt)
txt.append(empty_txt)

# do the same for first column and last columns
txt = ["." + t + "." for t in txt]

In [397]:
def is_weird_caracter(element):
    '''
    Returns True if the character in not a digic OR a dot.
    Finding a "weird" character means that we want to sum the number.
    '''
    return bool(re.compile(r'[^0-9.]').match(element))


def is_part_number_fun(current_row, prev_row, next_row, current_number):
    '''
    Check if a "weird" character is positioned around the current_number.
    If so, it means that we want to sum the number.
    '''
    pos = current_row.find(current_number)

    current_row_bool = (
        is_weird_caracter(current_row[pos-1]) |
        is_weird_caracter(current_row[pos+len(current_number)])
    )

    prev_next_row_bool = False
    for p in range(pos-1, pos+len(current_number)+1):
        prev_next_row_bool = (
            prev_next_row_bool |
            is_weird_caracter(prev_row[p]) |
            is_weird_caracter(next_row[p])
        )

    return current_row_bool | prev_next_row_bool
    

def replace_current_number_with_code(current_row, current_number):
    '''
    After we are done with a number (disregard if it had a weird character around or not),
    we want to transform it in a string of @.
    For ex: ..123.. --> ..@@@..
    This is done for not confusing a number already used with one still to be used.
    '''
    pattern = r'\b' + re.escape(current_number) + r'\b'
    match = re.search(pattern, current_row)
    pos = match.start()
    return current_row[:pos] + ("@" * len(current_number)) + current_row[pos+len(current_number):]

    
def sum_all_part_numbers_in_row(txt, r):
    '''
    Given a row, check for each number if it has weird character around.
    If yes we sum it, otherwise no.
    '''
    prev_row = txt[r-1]
    current_row = txt[r]
    next_row = txt[r+1]
    
    sum_part_numbers = 0

    for current_number in re.findall(r'\d+', current_row):
        if is_part_number_fun(current_row, prev_row, next_row, current_number):
            sum_part_numbers += int(current_number)
        current_row = replace_current_number_with_code(current_row, current_number)
    return sum_part_numbers

In [398]:
sum_part_numbers = 0

for r in range(1, len(txt)-1):
    sum_part_numbers += sum_all_part_numbers_in_row(txt, r)

print("Result:", sum_part_numbers)

Result: 522726


In [394]:
import math as m, re

board = list(open("/Users/adolci/Documents/Python/private/advent_of_code/2023/problem_3_input.txt"))
chars = {(r, c): [] for r in range(140) for c in range(140)
                    if board[r][c] not in '01234566789.'}

for r, row in enumerate(board):
    for n in re.finditer(r'\d+', row):
        edge = {(r, c) for r in (r-1, r, r+1)
                       for c in range(n.start()-1, n.end()+1)}

        for o in edge & chars.keys():
            chars[o].append(int(n.group()))

print(sum(sum(p)    for p in chars.values()),
      sum(m.prod(p) for p in chars.values() if len(p)==2))

522726 81721933


### --- Part Two ---

The missing part wasn't the only issue - one of the gears in the engine is wrong. A gear is any * symbol that is adjacent to exactly two part numbers. Its gear ratio is the result of multiplying those two numbers together.

This time, you need to find the gear ratio of every gear and add them all up so that the engineer can figure out which gear needs to be replaced.

Consider the same engine schematic again:

```
467..114..
...*......
..35..633.
......#...
617*......
.....+.58.
..592.....
......755.
...$.*....
.664.598..
```

In this schematic, there are two gears. The first is in the top left; it has part numbers 467 and 35, so its gear ratio is 16345. The second gear is in the lower right; its gear ratio is 451490. (The * adjacent to 617 is not a gear because it is only adjacent to one part number.) Adding up all of the gear ratios produces 467835.

What is the sum of all of the gear ratios in your engine schematic?